In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("../data/btcusdt_1h.csv", parse_dates=["open_time"])

df.head()



In [ ]:
plt.figure(figsize=(12, 5))
plt.plot(df['open_time'], df['close'], label='BTC Close Price')
plt.xlabel("Time")
plt.ylabel("Price (USD)")
plt.title("BTC/USDT Hourly Close Price")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# 1-hour return
df['return_1h'] = df['close'].pct_change()

# 3-period and 6-period moving averages (e.g., 3h and 6h)
df['ma_3'] = df['close'].rolling(window=3).mean()
df['ma_6'] = df['close'].rolling(window=6).mean()

# Rolling volatility (std deviation)
df['volatility_3'] = df['close'].rolling(window=3).std()

# Volume change
df['volume_change'] = df['volume'].pct_change()

In [ ]:
print(df.columns.tolist())
#df[['close', 'return_1h', 'ma_3', 'ma_6', 'volatility_3', 'volume_change']].tail(10)

In [ ]:
# Calculate % change between now and 24 hours later
df['future_return_24h'] = df['close'].shift(-24) / df['close'] - 1

# Create the binary label: 1 if price goes up, else 0
df['label'] = (df['future_return_24h'] > 0).astype(int)

df[['close', 'future_return_24h', 'label']].tail(30)

In [ ]:

from sklearn.model_selection import train_test_split

# Drop NaNs after all features and labels are created
df_clean = df.dropna()

# Select features and label
features = ['return_1h', 'ma_3', 'ma_6', 'volatility_3', 'volume_change']
X = df_clean[features]
y = df_clean['label']

# Time-based split (no shuffle)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False
)

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset

# Convert pandas DataFrames to PyTorch tensors
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)

X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

# Wrap in TensorDatasets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

In [ ]:
import torch.nn as nn
import torch.optim as optim

# Define the model
class CryptoClassifier(nn.Module):
    def __init__(self):
        super(CryptoClassifier, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(5, 64),  # 5 features
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 2)   # 2 output classes: up/down
        )

    def forward(self, x):
        return self.net(x)

# Instantiate the model
model = CryptoClassifier()

# Loss and optimizer
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 20
for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch + 1}/{epochs} - Loss: {total_loss:.4f}")

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Switch to evaluation mode
model.eval()

# Collect predictions and true labels
y_true = []
y_pred = []

with torch.no_grad():
    for batch_X, batch_y in test_loader:
        outputs = model(batch_X)
        _, predicted = torch.max(outputs, 1)
        y_true.extend(batch_y.tolist())
        y_pred.extend(predicted.tolist())

# Accuracy
accuracy = accuracy_score(y_true, y_pred)
print(f"Test Accuracy: {accuracy:.4f}")

# Classification report
print(classification_report(y_true, y_pred, target_names=["DOWN", "UP"]))

# Confusion matrix
cm = confusion_matrix(y_true, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=["DOWN", "UP"], yticklabels=["DOWN", "UP"])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
import torch
import numpy as np
from sklearn.utils.class_weight import compute_class_weight

# 1. Get the class labels
classes = np.unique(y_train.values)

# 2. Compute class weights using sklearn
weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train.values)

# 3. Convert weights to torch tensor
class_weights = torch.tensor(weights, dtype=torch.float32)

model = CryptoClassifier()
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 4. Use in loss function
criterion = nn.CrossEntropyLoss(weight=class_weights)

print("Class weights:", class_weights)

In [ ]:
torch.save(model.state_dict(), "model.pth")
